In [1]:
import h5py as h5
import netCDF4 as nc
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
import time
import cv2
import csv
from global_land_mask import globe 
import glob
import math
from tqdm import tqdm
import re
import seaborn as sns


In [6]:
# ust21 사이즈 
from scipy import io
mat_file = io.loadmat('/home/juneyonglee/Desktop/AY_ust/preprocessing/Land_mask/Land_mask.mat')
land_mask = mat_file['Land']

land_mask.shape


# GOCI 5685, 5567

(8000, 10500)

In [10]:
# modis 사이즈 
import h5py

def check_with_h5py(file_path):
    try:
        with h5py.File(file_path, 'r') as f:
            print("File opened successfully with h5py.")
            # 파일 내부의 키(데이터셋 구조) 출력
            print("File keys: ", list(f.keys()))
            # 필요한 경우 첫 번째 데이터셋의 세부 정보 확인
            first_key = list(f.keys())[0]
            dataset = f[first_key]
            print(f"Dataset '{first_key}' shape: {dataset.shape}")
            print(f"Dataset '{first_key}' data type: {dataset.dtype}")
    except OSError as e:
        print(f"Failed to open file with h5py: {e}")

# h5py로 파일 열기 시도
file_path = '/home/juneyonglee/Downloads/AQUA_MODIS.20140813_20140820.L3m.8D.CHL.chlor_a.4km.nc'
check_with_h5py(file_path)


File opened successfully with h5py.
File keys:  ['chlor_a', 'eightbitcolor', 'lat', 'lon', 'palette', 'processing_control', 'rgb']
Dataset 'chlor_a' shape: (4320, 8640)
Dataset 'chlor_a' data type: float32


In [31]:
import os
from PIL import Image

def check_image_loading(file_paths):
    for file_path in file_paths:
        try:
            # 이미지를 열어보기 시도
            with Image.open(file_path) as img:
                img.verify()  # 이미지 파일이 유효한지 확인
        except Exception as e:
            # 오류 발생 시 파일 경로와 오류 메시지 출력
            print(f"Loading error for: {file_path}")
            print(f"Error: {str(e)}")

# 파일 경로 리스트 예시 (newtrain 경로 내의 모든 .tiff 파일을 검색)
directory = "/media/juneyonglee/My Book/Preprocessed/GOCI/L2_Rrs_newtrain/band_2"
file_paths = []

# 디렉토리 내 모든 .tiff 파일 경로 추가
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith(".tiff"):
            file_paths.append(os.path.join(root, file))

# 이미지 로드 시도 및 오류 출력
check_image_loading(file_paths)


Loading error for: /media/juneyonglee/My Book/Preprocessed/GOCI/L2_Rrs_newtrain/band_2/0/RRS_band_2_nak_2011-07-15_r2445_c2948.tiff
Error: cannot identify image file '/media/juneyonglee/My Book/Preprocessed/GOCI/L2_Rrs_newtrain/band_2/0/RRS_band_2_nak_2011-07-15_r2445_c2948.tiff'
Loading error for: /media/juneyonglee/My Book/Preprocessed/GOCI/L2_Rrs_newtrain/band_2/0/RRS_band_2_nak_2011-07-15_r2428_c3033.tiff
Error: cannot identify image file '/media/juneyonglee/My Book/Preprocessed/GOCI/L2_Rrs_newtrain/band_2/0/RRS_band_2_nak_2011-07-15_r2428_c3033.tiff'
Loading error for: /media/juneyonglee/My Book/Preprocessed/GOCI/L2_Rrs_newtrain/band_2/0/RRS_band_2_nak_2015-03-26_r2208_c2926.tiff
Error: cannot identify image file '/media/juneyonglee/My Book/Preprocessed/GOCI/L2_Rrs_newtrain/band_2/0/RRS_band_2_nak_2015-03-26_r2208_c2926.tiff'
Loading error for: /media/juneyonglee/My Book/Preprocessed/GOCI/L2_Rrs_newtrain/band_2/0/RRS_band_2_nak_2015-03-26_r2208_c2927.tiff
Error: cannot identify im

In [32]:
# import os
# from PIL import Image

# def remove_error_files(file_paths):
#     for file_path in file_paths:
#         try:
#             # 이미지 열어보기 시도
#             with Image.open(file_path) as img:
#                 img.verify()  # 이미지 파일이 유효한지 확인
#         except Exception as e:
#             # 파일 열기에 실패하면 파일 삭제
#             print(f"Removing error file: {file_path}")
#             try:
#                 os.remove(file_path)
#             except Exception as remove_error:
#                 print(f"Failed to remove {file_path}: {str(remove_error)}")

# # 파일 경로 리스트 예시 (newtrain 경로 내의 모든 .tiff 파일을 검색)
# directory = "/media/juneyonglee/My Book/Preprocessed/GOCI/L2_Rrs_newtrain/band_2"
# file_paths = []

# # 디렉토리 내 모든 .tiff 파일 경로 추가
# for root, dirs, files in os.walk(directory):
#     for file in files:
#         if file.endswith(".tiff"):
#             file_paths.append(os.path.join(root, file))

# # 에러 발생 파일 삭제
# remove_error_files(file_paths)


Removing error file: /media/juneyonglee/My Book/Preprocessed/GOCI/L2_Rrs_newtrain/band_2/0/RRS_band_2_nak_2011-07-15_r2445_c2948.tiff
Removing error file: /media/juneyonglee/My Book/Preprocessed/GOCI/L2_Rrs_newtrain/band_2/0/RRS_band_2_nak_2011-07-15_r2428_c3033.tiff
Removing error file: /media/juneyonglee/My Book/Preprocessed/GOCI/L2_Rrs_newtrain/band_2/0/RRS_band_2_nak_2015-03-26_r2208_c2926.tiff
Removing error file: /media/juneyonglee/My Book/Preprocessed/GOCI/L2_Rrs_newtrain/band_2/0/RRS_band_2_nak_2015-03-26_r2208_c2927.tiff
Removing error file: /media/juneyonglee/My Book/Preprocessed/GOCI/L2_Rrs_newtrain/band_2/0/RRS_band_2_nak_2015-03-26_r2209_c2928.tiff
Removing error file: /media/juneyonglee/My Book/Preprocessed/GOCI/L2_Rrs_newtrain/band_2/0/RRS_band_2_nak_2015-03-26_r2209_c2939.tiff
Removing error file: /media/juneyonglee/My Book/Preprocessed/GOCI/L2_Rrs_newtrain/band_2/0/RRS_band_2_nak_2015-03-26_r2208_c2932.tiff
Removing error file: /media/juneyonglee/My Book/Preprocessed/G

In [ ]:
'''
   <UST21>
(unir) juneyonglee@juneyonglee-RTX3090-2:~/Desktop/AY_ust$ python preprocessing/nc_file_config.py
Variables: dict_keys([])
Dimensions: dict_keys(['number_of_lines', 'pixels_per_line', 'bands_per_pixel', 'number_of_reflectance_location_values', 'pixel_control_points', 'number_of_bands', 'number_of_reflective_bands'])
Groups: dict_keys(['sensor_band_parameters', 'scan_line_attributes', 'geophysical_data', 'navigation_data', 'processing_control'])
Group: /
  Dimension: number_of_lines 2030
  Dimension: pixels_per_line 1354
  Dimension: bands_per_pixel 16
  Dimension: number_of_reflectance_location_values 10
  Dimension: pixel_control_points 1354
  Dimension: number_of_bands 24
  Dimension: number_of_reflective_bands 16
  Group: /sensor_band_parameters
    Variable: wavelength (24,) int32
    Variable: vcal_gain (16,) float32
    Variable: vcal_offset (16,) float32
    Variable: F0 (16,) float32
    Variable: aw (16,) float32
    Variable: bbw (16,) float32
    Variable: k_oz (16,) float32
    Variable: k_no2 (16,) float32
    Variable: Tau_r (16,) float32
  Group: /scan_line_attributes
    Variable: year (2030,) int32
    Variable: day (2030,) int32
    Variable: msec (2030,) int32
    Variable: detnum (2030,) int8
    Variable: mside (2030,) int8
    Variable: slon (2030,) float32
    Variable: clon (2030,) float32
    Variable: elon (2030,) float32
    Variable: slat (2030,) float32
    Variable: clat (2030,) float32
    Variable: elat (2030,) float32
    Variable: csol_z (2030,) float32
  Group: /geophysical_data
    Variable: aot_869 (2030, 1354) int16
    Variable: angstrom (2030, 1354) int16
    Variable: Rrs_412 (2030, 1354) int16
    Variable: Rrs_443 (2030, 1354) int16
    Variable: Rrs_469 (2030, 1354) int16
    Variable: Rrs_488 (2030, 1354) int16
    Variable: Rrs_531 (2030, 1354) int16
    Variable: Rrs_547 (2030, 1354) int16
    Variable: Rrs_555 (2030, 1354) int16
    Variable: Rrs_645 (2030, 1354) int16
    Variable: Rrs_667 (2030, 1354) int16
    Variable: Rrs_678 (2030, 1354) int16
    Variable: chlor_a (2030, 1354) float32
    Variable: Kd_490 (2030, 1354) int16
    Variable: pic (2030, 1354) int16
    Variable: poc (2030, 1354) int16
    Variable: ipar (2030, 1354) int16
    Variable: nflh (2030, 1354) int16
    Variable: par (2030, 1354) int16
    Variable: l2_flags (2030, 1354) int32
  Group: /navigation_data
    Variable: longitude (2030, 1354) float32
    Variable: latitude (2030, 1354) float32
    Variable: cntl_pt_cols (1354,) int32
    Variable: cntl_pt_rows (2030,) int32
    Variable: tilt (2030,) float32
  Group: /processing_control
    Group: /processing_control/input_parameters
    Group: /processing_control/flag_percentages

    
<MODIS 8day 관련>
Variables: dict_keys(['chlor_a', 'lat', 'lon', 'palette'])
Dimensions: dict_keys(['lat', 'lon', 'rgb', 'eightbitcolor'])
Groups: dict_keys(['processing_control'])
Group: /
  Variable: chlor_a (4320, 8640) float32
  Variable: lat (4320,) float32
  Variable: lon (8640,) float32
  Variable: palette (3, 256) uint8
  Dimension: lat 4320
  Dimension: lon 8640
  Dimension: rgb 3
  Dimension: eightbitcolor 256
  Group: /processing_control
    Group: /processing_control/input_parameters

    <GOCI 관련>
 (unir) juneyonglee@juneyonglee-RTX3090-2:~/Desktop/AY_ust$ python preprocessing/nc_file_config.py
Variables: dict_keys([])
Dimensions: dict_keys([])
Groups: dict_keys(['HDFEOS', 'HDFEOS INFORMATION'])
Group: /
  Group: /HDFEOS
    Group: /HDFEOS/ADDITIONAL
      Group: /HDFEOS/ADDITIONAL/FILE_ATTRIBUTES
    Group: /HDFEOS/GRIDS
      Group: /HDFEOS/GRIDS/Image Data
        Group: /HDFEOS/GRIDS/Image Data/Data Fields
          Variable: Band 1 RRS Image Pixel Values (5685, 5567) float32
          Variable: Band 2 RRS Image Pixel Values (5685, 5567) float32
          Variable: Band 3 RRS Image Pixel Values (5685, 5567) float32
          Variable: Band 4 RRS Image Pixel Values (5685, 5567) float32
          Variable: Band 5 RRS Image Pixel Values (5685, 5567) float32
          Variable: Band 6 RRS Image Pixel Values (5685, 5567) float32
          Variable: Band 7 RRS Image Pixel Values (5685, 5567) float32
          Variable: Band 8 RRS Image Pixel Values (5685, 5567) float32
          Dimension: phony_dim_0 5685
          Dimension: phony_dim_1 5567
    Group: /HDFEOS/POINTS
      Group: /HDFEOS/POINTS/Ephemeris
        Group: /HDFEOS/POINTS/Ephemeris/Data
        Group: /HDFEOS/POINTS/Ephemeris/Linkage
      Group: /HDFEOS/POINTS/Event
        Group: /HDFEOS/POINTS/Event/Data
        Group: /HDFEOS/POINTS/Event/Linkage
      Group: /HDFEOS/POINTS/File Descripter Metadata
        Group: /HDFEOS/POINTS/File Descripter Metadata/Data
        Group: /HDFEOS/POINTS/File Descripter Metadata/Linkage
      Group: /HDFEOS/POINTS/Map Projection
        Group: /HDFEOS/POINTS/Map Projection/Data
        Group: /HDFEOS/POINTS/Map Projection/Linkage
      Group: /HDFEOS/POINTS/Navigation for GOCI
        Group: /HDFEOS/POINTS/Navigation for GOCI/Data
          Variable: Navigation for GOCI (128,) {'names': ['Band number', 'Slot number', 'Relative time', 'Spacecraft attitude', 'XO', 'YO', 'XS', 'YS', 'XPO', 'YPO', 'XPS', 'YPS', 'Number of valid A parameters', 'A parameters value', 'Number of valid B parameters', 'B parameters value', 'Number of valid C parameters', 'C parameters value', 'Number of valid D parameters', 'D parameters value', 'Number of valid A prime parameters', 'A prime parameters value', 'Number of valid B prime parameters', 'B prime parameters value', 'Number of valid C prime parameters', 'C prime parameters value', 'Number of valid D prime parameters', 'D prime parameters value'], 'formats': ['<i4', '<i4', '<f4', ('<f4', (3,)), '<f4', '<f4', '<f4', '<f4', '<f4', '<f4', '<f4', '<f4', '<i4', ('<f4', (16,)), '<i4', ('<f4', (16,)), '<i4', ('<f4', (16,)), '<i4', ('<f4', (16,)), '<i4', ('<f4', (16,)), '<i4', ('<f4', (16,)), '<i4', ('<f4', (16,)), '<i4', ('<f4', (16,))], 'offsets': [0, 4, 8, 12, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 124, 128, 192, 196, 260, 264, 328, 332, 396, 400, 464, 468, 532, 536], 'itemsize': 600, 'aligned': True}
          Dimension: phony_dim_2 128
        Group: /HDFEOS/POINTS/Navigation for GOCI/Linkage
      Group: /HDFEOS/POINTS/Radiometric Calibration for GOCI
        Group: /HDFEOS/POINTS/Radiometric Calibration for GOCI/Data
        Group: /HDFEOS/POINTS/Radiometric Calibration for GOCI/Linkage
      Group: /HDFEOS/POINTS/Scene Header
        Group: /HDFEOS/POINTS/Scene Header/Data
        Group: /HDFEOS/POINTS/Scene Header/Linkage
      Group: /HDFEOS/POINTS/Validataion (trailer)
        Group: /HDFEOS/POINTS/Validataion (trailer)/Data
        Group: /HDFEOS/POINTS/Validataion (trailer)/Linkage
  Group: /HDFEOS INFORMATION
    Variable: StructMetadata.0 () <class 'str'>
'''